### EXERCICI 1

Descarrega el data set Airlines Delay: Airline on-time statistics and delay causes i carrega’l a un Pandas Dataframe. Explora les dades que conté, explica breument quines variables hi ha i queda’t únicament amb les columnes que consideris rellevants. Justifica la teva elecció.

Redueix la dimensió del dataset de manera aleatòria per tal d'obtenir un dataset de només 200.000 registres. Tots els exercicis s'han de fer amb aquest dataset reduït.

In [1]:
import pandas as pd

pd.options.display.max_rows = 10
pd.options.display.max_columns = None

df = pd.read_csv('DelayedFlights.csv')
df = df.sample(n=200000)

'''Hi ha molts de NaNs a les columnes de les causes dels delays. Això és degut a que hi ha vols 
que no han tingut delays. Filtrarem la taula per a que només apareguin els vols endarrerits(ArrDelay > 0). També exclourem
els cancelats i els desviats, ja que aquests últims alteren les dades.'''

df_neta = df[(df["Cancelled"] == 0) & (df["Diverted"] == 0) & (df["ArrDelay"] > 0)]

'''Una vegada fet això, seleccionarem les variables més importants, com són les de les causes dels delays, les rutes,
les aerolínies...'''

df_def = df_neta[['UniqueCarrier','AirTime', 'Distance', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'CarrierDelay', 
                  'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']]

print(df_def)



        UniqueCarrier  AirTime  Distance  ArrDelay  DepDelay Origin Dest  \
1065809            B6    118.0       944       7.0      30.0    MCO  JFK   
26867              WN     44.0       256      11.0      13.0    LAS  PHX   
612461             YV     53.0       347      27.0      23.0    PHX  ELP   
1185477            EV     55.0       357       6.0       6.0    ROA  ATL   
1176262            DL    400.0      3110      22.0      14.0    CVG  ANC   
...               ...      ...       ...       ...       ...    ...  ...   
1274814            WN     70.0       377     291.0     300.0    MHT  BWI   
725568             DL    119.0       874      46.0      37.0    SAT  ATL   
1915650            B6     64.0       387     166.0     161.0    SMF  LGB   
986932             DL    123.0       859       6.0       9.0    ATL  BDL   
1356895            EV     56.0       433       3.0      12.0    ATL  EWN   

         CarrierDelay  WeatherDelay  NASDelay  SecurityDelay  \
1065809           NaN  

### EXERCICI 2

### Fes un informe complet del dataset:

#### - Resumeix estadísticament el dataset i les columnes d’interès. Fes una anàlisi estadístic del que consideris rellevant.

In [2]:
TotalCarrierDelay = df_def['CarrierDelay'].sum()
print('TotalCarrierDelay = ', TotalCarrierDelay.astype(int))

TotalWeatherDelay = df_def['WeatherDelay'].sum()
print('TotalWeatherDelay = ', TotalWeatherDelay.astype(int))

TotalNASDelay = df_def['NASDelay'].sum()
print('TotalNASDelay = ', TotalNASDelay.astype(int))

TotalSecurityDelay = df_def['SecurityDelay'].sum()
print('TotalSecurityDelay = ', TotalSecurityDelay.astype(int))

TotalLateAircraftDelay = df_def['LateAircraftDelay'].sum()
print('TotalLateAircraftDelay = ', TotalLateAircraftDelay.astype(int))

TotalCarrierDelay =  2464318
TotalWeatherDelay =  479143
TotalNASDelay =  1946022
TotalSecurityDelay =  10504
TotalLateAircraftDelay =  3265946


In [3]:
MaxTotalDelay = max(TotalCarrierDelay, TotalWeatherDelay, TotalNASDelay, TotalSecurityDelay, TotalLateAircraftDelay)
print(MaxTotalDelay.astype(int))

3265946


Com podem veure, la causa més habitual dels endarreriments és LateAircraftDelay, que és un endarreriment produït perquè
l'anterior vol ha arribat tard, generant un efecte dominó.

#### - Troba quantes dades faltants hi ha per columna.

In [4]:
faltants = df_def.isnull().sum().reset_index()
faltants = faltants.rename(columns={'index': 'variables', 0: 'quantitat'})


print(faltants)

            variables  quantitat
0       UniqueCarrier          0
1             AirTime          0
2            Distance          0
3            ArrDelay          0
4            DepDelay          0
..                ...        ...
7        CarrierDelay      49154
8        WeatherDelay      49154
9            NASDelay      49154
10      SecurityDelay      49154
11  LateAircraftDelay      49154

[12 rows x 2 columns]


#### - Crea columnes noves (velocitat mitjana del vol, si ha arribat tard o no...).

In [5]:
#'Airtime' és en minuts i 'Distance' és en milles, s'ha de convertir a km/h i eliminar els valors nulls.

df_def = df_def.dropna(subset=['AirTime', 'Distance'])
df_def['VelMitjana(km/h)'] = df_def['Distance']/df_def['AirTime'] * 1.6 * 60
print(df_def['VelMitjana(km/h)'].head(10).astype(int))

print()

df_def = df_def.dropna(subset=['ArrDelay'])
df_def['Endarrerit'] = df_def['ArrDelay'] > 0

print(df_def['Endarrerit'].head(10))

1065809    768
26867      558
612461     628
1185477    623
1176262    746
1657952    391
1811298    923
1039483    641
598058     481
44207      400
Name: VelMitjana(km/h), dtype: int32

1065809    True
26867      True
612461     True
1185477    True
1176262    True
1657952    True
1811298    True
1039483    True
598058     True
44207      True
Name: Endarrerit, dtype: bool


#### - Fes una taula de les aerolínies amb més endarreriments acumulats.

In [6]:
aerolinees_endererrides = df_def.groupby('UniqueCarrier')['ArrDelay'].sum().astype(int)
aerolinees_endererrides = aerolinees_endererrides.sort_values(ascending=False)
aerolinees_endererrides = aerolinees_endererrides.reset_index(name="TotalMinuts")

pd.options.display.max_rows = 20

print(aerolinees_endererrides)

   UniqueCarrier  TotalMinuts
0             WN      1205176
1             AA       920680
2             UA       698256
3             MQ       665650
4             OO       608358
5             XE       549051
6             DL       482085
7             CO       440005
8             EV       405310
9             US       392168
10            YV       390907
11            NW       341520
12            FL       322204
13            B6       315596
14            OH       269977
15            9E       262082
16            AS       158171
17            F9        84719
18            HA        27997
19            AQ         1970


#### - Quins són els vols més llargs? I els més endarrerits? Busca les rutes més llargues i les que acumulen més retards.

In [7]:
print("Rutes més llargues:\n")
print(df_def[['Origin','Dest','Distance']].drop_duplicates('Distance').sort_values('Distance', ascending=False).head(10))


rutes_mes_endarrerides = df_def.groupby(['Origin', 'Dest'])['ArrDelay'].sum().astype(int)
rutes_mes_endarrerides = rutes_mes_endarrerides.reset_index(name="TotalArrDelay")
rutes_mes_endarrerides = rutes_mes_endarrerides.sort_values(by="TotalArrDelay", ascending=False)

print("\nRutes més endarrerides:\n")
print(rutes_mes_endarrerides.head(10))


Rutes més llargues:

        Origin Dest  Distance
871328     EWR  HNL      4962
571938     ATL  HNL      4502
1464101    HNL  ORD      4243
1147069    KOA  ORD      4213
1147089    ORD  OGG      4184
1381167    HNL  MSP      3972
1254820    HNL  IAH      3904
1502153    DFW  HNL      3784
691537     DFW  OGG      3711
1082050    ANC  ATL      3417

Rutes més endarrerides:

     Origin Dest  TotalArrDelay
3466    ORD  LGA          25878
2581    LGA  ORD          23564
2509    LAX  SFO          23354
3437    ORD  EWR          22722
4258    SFO  LAX          21761
168     ATL  EWR          20475
1634    EWR  ORD          20430
207     ATL  LGA          20316
1581    EWR  ATL          20248
2544    LGA  ATL          17046


#### - Aporta allò que consideris rellevant.

Amb totes aquestes dades podriem arribar a crear models de visualització per a veure patrons i tendències i trobar una solució parcial -no total, ja que sempre que existeixi factor humà serà impossible- als endarreriments. Amb les variables de les que disposem, podem veure que hi ha rutes on hi ha molts més enderreriments que a altres. Això es podria investigar per a saber si és degut, per exemple, a que per a determinades rutes només existeixen vols de certes companyies i que aquestes són, casualment, les que més fallen en quant a puntualitat (poc probable) o es deu a altres factors que s'haurien d'analitzar en profunditat.

### EXERCICI 3

Exporta el dataset net i amb les noves columnes a Excel.

In [8]:
df_def.to_excel('exercici_delays.xlsx')